# Tutorial 1: Audio Generation

<p align="right" style="margin-right: 8px;">
    <a target="_blank" href="https://colab.research.google.com/github/idiap/sdialog/blob/main/tutorials/01_audio/1.audio_generation.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p>

## Getting started

### Environment Setup

Let's first check if our environment is all set up.

In [ ]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
import os
from IPython import get_ipython
from IPython.display import Audio, display

if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    # Installing sdialog
    !git clone https://github.com/idiap/sdialog.git
    %cd sdialog
    %pip install -e .[audio]
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    get_ipython().system = os.system

### Load the example dialogue

In order to run the next steps in a fast manner, we will start from an existing dialog generated using previous tutorials:

In [ ]:
from sdialog import Dialog

path_dialog = "../../tests/data/demo_dialog_doctor_patient.json"

if not os.path.exists(path_dialog) and not os.path.exists("./demo_dialog_doctor_patient.json"):
    !wget https://raw.githubusercontent.com/idiap/sdialog/refs/heads/main/tests/data/demo_dialog_doctor_patient.json
    path_dialog = "./demo_dialog_doctor_patient.json"

original_dialog = Dialog.from_file(path_dialog)
original_dialog.print()

# Tutorial

### Instanciate voices database

In [ ]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
kokoro_voice_database = HuggingfaceVoiceDatabase("sdialog/voices-kokoro")

### Instanciate TTS model

In [ ]:
!pip install -q kokoro>=0.9.4
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

In [ ]:
from sdialog.audio.tts_engine import KokoroTTS
tts_engine = KokoroTTS()

You can also try later on some TTS models from the HuggingFace HUB if you want by simply using:

In [ ]:
%%script false --no-raise-error
from sdialog.audio.tts_engine import HuggingFaceTTS
tts_engine = HuggingFaceTTS("facebook/mms-tts-eng")  # or any other model from the HuggingFace HUB

## Setup stage: Audio Dialog and Audio Pipeline

In [ ]:
from sdialog.audio.dialog import AudioDialog
from sdialog.audio.pipeline import AudioPipeline

Convert the original dialog into a audio enhanced dialog

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

Identify speakers names:

In [ ]:
print("Speaker 1:", dialog.speakers_names["speaker_1"])
print("Speaker 2:", dialog.speakers_names["speaker_2"])

## Concatenated utterances with no room accoustics (also called step 1)

Instanciate the audio pipeline in order to use `Kokoro` (`tts_engine`) as the TTS model and save the outputs of all the dialogs into the directory `./audio_outputs`.

The voices are sampled from the `kokoro_voice_database` based on the persona attributes `age`, `gender` and `language`, as assigned during the original textual dialog.

In [ ]:
os.makedirs("./audio_outputs", exist_ok=True)

Can be used with default values:

In [ ]:
audio_pipeline = AudioPipeline()
dialog: AudioDialog = audio_pipeline.inference(dialog)

In [ ]:
display(Audio(dialog.audio_step_1_filepath, autoplay=True, rate=24000))

It will select Kokoro as the default TTS model (with officials voices), save the audio files in `outputs` and run only the step 1 of the pipeline (no room accoustics).

Or by specifying more or less parameters:

In [ ]:
audio_pipeline = AudioPipeline(
    voice_database=kokoro_voice_database,
    tts_pipeline=tts_engine,
    dir_audio="./audio_outputs",
)

Perform the inference of the audio pipeline on the previously converted dialog. In this case we will focus on generating the "unprocessed" audio, which consist of the agregation of all utterances from the dialog. Rather than using the dialog identifier as the name of the directory, we are using here a custom directory name `demo_dialog_kokoro` which will be saved at `./audio_outputs/demo_dialog_kokoro/`. 

In [ ]:
# Generate the audio for the dialog
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    dialog_dir_name="demo_dialog_kokoro",
)

# Path to the audio of the first stage of the audio pipeline
print("Audio generated successfully at:", dialog.audio_step_1_filepath)

In [ ]:
display(Audio(dialog.audio_step_1_filepath, autoplay=True, rate=24000))

## Let's do the same but now by attributing specific voices to the speakers 

In [ ]:
from sdialog.audio.utils import Role

In [ ]:
# Generate the audio for the dialog
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    dialog_dir_name="demo_dialog_kokoro_selected_voices",
    voices={
        Role.SPEAKER_1: ("am_michael","english"),
        Role.SPEAKER_2: ("af_bella","english"),
    }
)

# Path to the audio of the first stage of the audio pipeline
print("Audio generated successfully at:", dialog.audio_step_1_filepath)

In [ ]:
display(Audio(dialog.audio_step_1_filepath, autoplay=True, rate=24000))

## Generate an audio for a dialogue in one function call 🤯

You can also use Dialog internal function to convert the dialogue into a AudioDialog with the audio files:

In [ ]:
new_audio_dialog = original_dialog.to_audio()

In [ ]:
display(Audio(new_audio_dialog.audio_step_1_filepath, autoplay=True, rate=24000))

Or by using the utility function `to_audio` which share the same parameters as the internal method to the Dialog object:

In [ ]:
from sdialog.audio.pipeline import to_audio

In [ ]:
new_audio_dialog = to_audio(
    original_dialog,
    audio_file_format="mp3", # can also be generated with mp3 / wav / flac formats
    re_sampling_rate=16000,
    dialog_dir_name="utility_function_demo"
)

In [ ]:
display(Audio(new_audio_dialog.audio_step_1_filepath, autoplay=True, rate=16000))